# PVML NIM integration
## Prerequisites

Before running this notebook, ensure you have:
- PVML SDK installed (`pip install -r requirements.txt`)
- Access to a PVML workspace with a datasource and view configured


In [ ]:
import os
from pathlib import Path

from dotenv import load_dotenv


# Load environment variables from .env.local
def load_env_variables(env_file: str = ".env") -> bool:
    """Load environment variables from .env.local file using python-dotenv"""
    env_path = Path(env_file)

    if not env_path.exists():
        print(f"Warning: {env_file} not found. Please create it with your PVML credentials.")
        print("\nRequired format:")
        print("PVML_API_KEY=your_api_key_here")
        print("PVML_API_URL=your_api_url_here")
        print("GPT_KEY=your_gpt_key_here")
        print("PVML_WORKSPACE_ID=your_workspace_id_here")
        return False

    # Load environment variables from .env.local
    load_dotenv(env_path)
    print(f"Environment variables loaded from {env_file}")
    return True


# Load environment variables
env_loaded = load_env_variables()

# Validate required environment variables
required_vars = ["PVML_API_URL"]
missing_vars = [var for var in required_vars if not os.getenv(var)]

if missing_vars:
    raise ValueError(f"Missing required environment variables: {missing_vars}")


In [24]:
PVML_API_KEY = "YOUR_PVML_API_KEY"
PVML_WORKSPACE_ID = "YOUR_PVML_WORKSPACE_ID"
PVML_VIEW_ID = "YOUR_PVML_VIEW_ID"
NIM_API_KEY = "NIM_API_KEY"


In [ ]:
# Import PVML SDK and Initialize Client
from pvml import Client as PVMLClient

pvml_client = PVMLClient(api_key=PVML_API_KEY)
print("Client ready for use")

In [26]:
# Retrieve the workspace
workspace = pvml_client.get_workspace(workspace_id=PVML_WORKSPACE_ID)

In [27]:
# Retrieve the relevant view
secure_view = workspace.get_view(view_id=PVML_VIEW_ID)
db_mcp_id = secure_view.get_mcp_id()

In [28]:
# NIM config
nim_model_name = "meta/llama-3.1-70b-instruct"
nim_base_url = "https://integrate.api.nvidia.com/v1"

In [29]:
# Create Secure agent using NIM
from pvml import LLM
nim_props = LLM.create_nim_props(nim_model_name, nim_base_url)
nim_llm = workspace.create_llm("nim_integration", "the llm powering the secure MCP", "nvidia", "nim",
                               NIM_API_KEY, nim_props)

In [30]:
# Create secure DB agent
prompts = pvml_client.get_default_prompts()
secure_agent = workspace.create_agent(name="secure nim agent", prompt=prompts['prompt'], 
                                      description="Secure db retrival", mcp_ids=[db_mcp_id], llm=nim_llm)

In [ ]:
# Query the secure agent
response_stream = secure_agent.generate("hi")
for event in response_stream:
    print(event)